# take a video on webcam

```
cd ../src/utils
python save_img.py
```

# Extract Frames from Video File

In [14]:
from pathlib import Path


save_dir = Path('../data/target/')
try: 
    save_dir.mkdir(exist_ok=True)
except FileExistsError : 
    print("Directory exists. Abort creation")
    


img_dir = save_dir.joinpath('images')
try:
    img_dir.mkdir(exist_ok=True)
except FileExostsError:
    print("Directory exists. Abort creation")

In [1]:
import cv2 
import numpy as np

def extract_images_from_video(filepath, subsample =  1):
    """
    filepath: string
        file path to video.
    subsample: int 
        use only one of subsample images.
        subsample = 1, use all images.
    
    """
    
    print("Video path: ",str(save_dir.joinpath(filepath)))
    cap = cv2.VideoCapture(str(save_dir.joinpath(filepath)))
    crop = True
    i = 0
    j = 0
    flag = None
    while(cap.isOpened()):
        #Extract only 1 of subsample images
        flag, frame = cap.read()
        if flag == False:
            break
        if j%subsample == 0:

            # Crop images to desired size 
            if crop:
                frame = frame[355:-10,645:-645,:]
            #frame = cv2.resize(frame, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)

            #if i >= 10:
            #    break

            cv2.imwrite(str(img_dir.joinpath('img_{:05d}.png'.format(i))), frame)
            i += 1


        j+= 1
    print("Number of frames:", i)

# Pose estimation (OpenPose)

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from pathlib import Path

%matplotlib inline

In [5]:
openpose_dir = Path('../src/pytorch_Realtime_Multi-Person_Pose_Estimation/')

import sys
sys.path.append(str(openpose_dir))
sys.path.append('../src/utils')



### Autorelaod modules 

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
# openpose
from network.rtpose_vgg import get_model
from evaluate.coco_eval import get_multiplier, get_outputs

# utils
from openpose_utils import remove_noise, get_pose

In [8]:
weight_name = openpose_dir.joinpath('network/weight/pose_model.pth')

model = get_model('vgg19')     
model.load_state_dict(torch.load(weight_name))
model = torch.nn.DataParallel(model).cuda()
model.float()
model.eval()
pass

Bulding VGG19


## check

In [ ]:
img_path = sorted(img_dir.iterdir())[0]
img = cv2.imread(str(img_path))
shape_dst = np.min(img.shape[:2])
# offset
oh = (img.shape[0] - shape_dst) // 2
ow = (img.shape[1] - shape_dst) // 2

img = img[oh:oh+shape_dst, ow:ow+shape_dst]
img = cv2.resize(img, (512, 512))
          
plt.imshow(img[:,:,[2, 1, 0]]) # BGR -> RGB

In [ ]:
multiplier = get_multiplier(img)
with torch.no_grad():
    paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    
r_heatmap = np.array([remove_noise(ht)
                      for ht in heatmap.transpose(2, 0, 1)[:-1]])\
                     .transpose(1, 2, 0)
heatmap[:, :, :-1] = r_heatmap
param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
label = get_pose(param, heatmap, paf)

plt.imshow(label)

## make label images for pix2pix

In [ ]:
train_dir = save_dir.joinpath('train')
try:
    train_dir.mkdir(exist_ok=True)
except FileExistsError:
    print("Directory exists. Abort creation")
    

train_img_dir = train_dir.joinpath('train_img')
train_img_dir.mkdir(exist_ok=True)
train_label_dir = train_dir.joinpath('train_label')
train_label_dir.mkdir(exist_ok=True)

for idx in tqdm(range(20)):
    img_path = img_dir.joinpath('img_{:05d}.png'.format(idx))
    img = cv2.imread(str(img_path))
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2

    img = img[oh:oh+shape_dst, ow:ow+shape_dst]
    img = cv2.resize(img, (512, 512))
    multiplier = get_multiplier(img)
    with torch.no_grad():
        paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    r_heatmap = np.array([remove_noise(ht)
                      for ht in heatmap.transpose(2, 0, 1)[:-1]])\
                     .transpose(1, 2, 0)
    heatmap[:, :, :-1] = r_heatmap
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    label = get_pose(param, heatmap, paf)
    
    cv2.imwrite(str(train_img_dir.joinpath('img_{:05d}.png'.format(idx))), img)    
    cv2.imwrite(str(train_label_dir.joinpath('label_{:05d}.png'.format(idx))), label)
    
torch.cuda.empty_cache()